In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=7908d05dfc5be40628ae2d86ef5c422da6b3893d5a402d8ac991b4e729d81b8f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=2eaea65fc34194f25bfff8b78924c9dc879fd7bbb9cb51e9df811e484196d50c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.4.1


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
      kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
      activation='relu',
      input_shape=(28,28,1)
   ),
   keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
      kernel_size=hp.Choice('conv_s_kernel', values = [3,5]),
      activation='relu'
   ),
   keras.layers.Flatten(),
   keras.layers.Dense(
      units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
      activation='relu'
   ),
   keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 28s]
val_accuracy: 0.9125000238418579

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 02m 34s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,647,242
Trainable params: 2,647,242
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1521 - accuracy: 0.9426 - val_loss: 0.2504 - val_accuracy: 0.9125
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1100 - accuracy: 0.9582 - val_loss: 0.2690 - val_accuracy: 0.9150
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0848 - accuracy: 0.9689 - val_loss: 0.3149 - val_accuracy: 0.9135
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0618 - accuracy: 0.9774 - val_loss: 0.3403 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0477 - accuracy: 0.9828 - val_loss: 0.3822 - val_accuracy: 0.9148
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0351 - accuracy: 0.9872 - val_loss: 0.4596 - val_accuracy: 0.9037
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0329 - accuracy: 0.9883 - val_loss: 0.5005 - val_accuracy: